# Konvertering av Bergens registeret



In [1]:
import polars as pl

In [2]:
'''
Medlemsnummer intervall fra 11000 til 16000.
Medlemsnummer i bergens registeret strekker seg mellom 1 og 6000
I Cornerstone er det ledige nummer intervall mellom 11000 og 16000
For å få de gamle medlemsnummer innenfor intervallet så plusser vi 11000 til gammelt medlemsnummer
Bergen hadde 282 medlemme ved årskiftet til 2025
'''
RANGE_START: int = 11000

In [3]:
# leser inn Data exportert i fra Bergens Access database
# I fra den tabellen som heter "tblInnmeldte"
df = (pl.read_excel("tblInnmeldte 2025-06-12.xlsx")
    # Vet ikke hvorfor dette blir gjort, men det er det som skjer i access databasen "qryMedlemmer" -
    # som er utgangspunkt for alle data som vises i Access
    # uten filter finnes 333 medlemmer, mens med filter finnes 284 medlemmer.
    # det er disse som har vært utgangspunkt for medlems opptellig
    .filter(pl.col("POSTNR").is_between(pl.lit("100"), pl.lit("9000"), closed="both"))
)
df

Could not determine dtype for column 41, falling back to string
Could not determine dtype for column 42, falling back to string


MedlemID,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,Dåpsdato,Ant barn under 15 år,Dåpskirke/sted,Fødselsdato,Personnummer,EFTERNAVN,FORNAVN,Stilling,ADRESSE,E-postadresse,POSTNR,Og,SAM_FOR,SAM_EFTER,Utsendelse,Utsendelses kode,Telefon priv,Telefon mob,Sivilstand,Trossamfunn,Gift?,Utsendelsesformat,Ektemakens trossamfunn,Anmerkning,Kategori,Giro,RegDato,SamID,FarID,MorID,Kjønn,Svart?,Faddere,Er mor og far gift?,Prest,Dåpsattest,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr
i64,date,bool,str,str,date,i64,str,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,bool,datetime[ms],i64,i64,i64,str,bool,str,bool,str,bool,str,str,date,str,str
4653,null,false,null,null,null,null,null,null,null,"""Hauger""","""E.""",null,"""_x000D_ Nordahl Rolfsensvei 3…",null,"""5094""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,null,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4654,null,false,null,null,null,null,null,null,null,"""Haugger""","""Elisabeth""",null,null,null,"""5094""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,null,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4655,null,false,null,null,null,null,null,null,null,"""Hauger""","""Elisabeth""",null,"""_x000D_ Nordahl Rolfsensvei 3…",null,"""5094""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,2024-12-04 17:24:32,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4656,null,false,null,null,null,null,null,null,null,"""Hauger""","""Elisabeth""",null,"""_x000D_ Nordahl Rolfsensvei 3…",null,"""5094""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,2024-12-04 17:25:45,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4628,null,false,"""Bergen""","""Bergen""",null,2,null,1992-10-19,"""19109245731""","""Tangen""","""Andreas""",null,"""Wergelandsalléen 18""",null,"""5067""",null,null,null,null,null,null,null,"""Gift mann""","""Kristensamfunnet""",false,"""E""","""Kristensamfunnet""",null,"""M""",false,2023-06-12 10:14:29,2248,null,null,"""M""",false,null,false,null,false,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4663,2024-12-11,false,null,null,null,null,null,1945-09-02,"""02094546322""","""Bjermeland""","""Trygve""",null,"""Munkebotn 33""",null,"""5036""",null,null,null,null,null,null,null,null,null,false,null,null,null,"""M""",false,2025-02-16 20:09:32,null,null,null,"""M""",false,null,false,null,false,null,null,null,null,null
4664,null,false,null,null,null,null,null,null,null,"""Ruscheinski""","""Gerd Peter""",null,"""Kråkenesdalen 191""",null,"""5153""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,2025-05-18 00:00:00,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4665,null,false,null,null,null,null,null,null,null,"""Ruscheinski""","""Gerd Peter""",null,"""Kråkenesdalen 191""",null,"""5153""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,2025-05-31 19:40:13,null,null,null,null,false,null,false,null,false,null,null,null,null,null


In [4]:
# Oppdatering av medlemsID til Cornerstone standard og litt vasking også
df = df.with_columns(
    # Lager nye medlemsnummere som er tilpasset Cornerstone
    (pl.col("MedlemID") + RANGE_START).alias("MedlemID"),
    (pl.col("SamID") + RANGE_START).alias("SamID"),
    (pl.col("MorID") + RANGE_START).alias("MorID"),
    (pl.col("FarID") + RANGE_START).alias("FarID"),
    # Tar bort Personnummer der det bare er skervet in fødselsdato delen
    # Har sjekket at "Fødselsdato" er satt på alle det gjelder
    pl.when(pl.col("Personnummer").str.len_chars() == 6)
      .then(None)
      .otherwise(pl.col("Personnummer"))
      .alias("Personnummer")
)


In [5]:
df.select("MedlemID", "SamID", "FORNAVN", "EFTERNAVN", "Personnummer"
         ).sort("SamID"
               )#.filter(pl.col("Personnummer").str.len_chars() == 6)

MedlemID,SamID,FORNAVN,EFTERNAVN,Personnummer
i64,i64,str,str,str
15653,null,"""E.""","""Hauger""",null
15654,null,"""Elisabeth""","""Haugger""",null
15655,null,"""Elisabeth""","""Hauger""",null
15656,null,"""Elisabeth""","""Hauger""",null
15619,null,"""Vårin Tangen""","""Lunde""",null
…,…,…,…,…
15547,15604,"""Tone Bell""","""Rysst""","""22055032878"""
15551,15606,"""Tora Døssland""","""Eikeland""","""16065247058"""
13248,15628,"""Katarina""","""Lunde""","""03089142074"""


## Rekkefølge på konverteringen
- Alle som har fødselsnummer, medlemmer med flere
- Alle som er registrert med "MedlemID", "SamID", "MorID", "FarID"
- Alle som har "MedlemID" i "SamID", "MorID", "FarID"
- 
- Det vil være overlappende "MedlemID" for de som er listet over. Få ut en liste og krymp den til disticte verdier av "MedlemID"
- Resterende personer vil ha datakvalitet av varierende kvalitet. Disse konverteres til slutt. Det er resterende som ikke er med i den oppbyggde "MedlemID" listen. Ingen av disse er medlemmer. De har heller ikke forbindelser til andre personer.

### Denne er ikke nødvendig, så lenge vi gjør den siste testen ved henting av data fra excel filen
``` python
df_medlemId = (
    df
    .filter(
        (pl.col("Kategori").str.starts_with("M")) &
        # Vet ikke hvorfor dette blir gjort, men det er det som skjer i access databasen "qryMedlemmer"
        (pl.col("POSTNR").is_between(pl.lit("100"), pl.lit("9000"), closed="both"))
    )
    .select("MedlemID")
)
df_medlemId
```

In [6]:
# Henter ut alle personer som har andre personer knyttet til seg
# Fornavn og etternavn hentes også på personen som har tilknyttninger
df_connectedMedlemer = pl.concat([
    df.filter(pl.col(c).is_not_null())
      .select(
          pl.col("FORNAVN"),
          pl.col("EFTERNAVN"),
          pl.lit(c).alias("type"),
          pl.col("MedlemID"),
          pl.col(c).alias("ConnectedMedlemID")
      )
    for c in ["SamID", "MorID", "FarID"]
])
df_connectedMedlemer

FORNAVN,EFTERNAVN,type,MedlemID,ConnectedMedlemID
str,str,str,i64,i64
"""Andreas""","""Tangen""","""SamID""",15628,13248
"""Sissel""","""Monsen""","""SamID""",15513,15484
"""Marit""","""Brendbekken""","""SamID""",14476,14477
"""Stein""","""Befring""","""SamID""",14477,14476
"""Per Gøran""","""Krüger""","""SamID""",14503,11213
…,…,…,…,…
"""Mari Ulevik""","""Furubotn""","""FarID""",15450,15479
"""Ingrid Kristine Miljeteig""","""Geelmuyden""","""FarID""",15452,15451
"""Erik Andreas""","""Geelmuyden""","""FarID""",15453,15451


In [7]:
# Finner alle ID'er som ikke eksisterer.
# Her er det feil i dato og dette er en del av vasken

# 1. Anti-join: IDs in connected that aren't in df
missing_ids = df_connectedMedlemer.join(
    df, 
    left_on="ConnectedMedlemID",      # Key in the left DataFrame
    right_on="MedlemID",              # Key in the right DataFrame
    how="anti"
).select(pl.col("ConnectedMedlemID").unique().alias("MedlemID"))
print(missing_ids)

# Sletter alle ID'er som ikke finnes fra vår Connected liste
# Sletter alle koblinger til personer som ikke finnes i registeret
df_connectedMedlemer = df_connectedMedlemer.join(
    missing_ids,
    left_on="ConnectedMedlemID",      # Key in the left DataFrame
    right_on="MedlemID",              # Key in the right DataFrame
    how="anti"
)
print(df_connectedMedlemer)

shape: (35, 1)
┌──────────┐
│ MedlemID │
│ ---      │
│ i64      │
╞══════════╡
│ 11025    │
│ 11112    │
│ 11113    │
│ 11212    │
│ 11215    │
│ …        │
│ 14743    │
│ 15444    │
│ 15445    │
│ 15485    │
│ 15533    │
└──────────┘
shape: (381, 5)
┌───────────────────────────┬─────────────┬───────┬──────────┬───────────────────┐
│ FORNAVN                   ┆ EFTERNAVN   ┆ type  ┆ MedlemID ┆ ConnectedMedlemID │
│ ---                       ┆ ---         ┆ ---   ┆ ---      ┆ ---               │
│ str                       ┆ str         ┆ str   ┆ i64      ┆ i64               │
╞═══════════════════════════╪═════════════╪═══════╪══════════╪═══════════════════╡
│ Andreas                   ┆ Tangen      ┆ SamID ┆ 15628    ┆ 13248             │
│ Sissel                    ┆ Monsen      ┆ SamID ┆ 15513    ┆ 15484             │
│ Marit                     ┆ Brendbekken ┆ SamID ┆ 14476    ┆ 14477             │
│ Stein                     ┆ Befring     ┆ SamID ┆ 14477    ┆ 14476             │
│

In [8]:

# Tror ikke dette blir nødvendig. Personers forbindelser blir oppdatert etter at alle
# relevante personer er lagt inn i egen ferdig vasket tabell under "df_connectedMedlemer"

# Oppdaterer df og setter alle ikke eksisterende kontakter til null
#df = df.with_columns([
#    pl.when(pl.col(c).is_in(missing_ids["MedlemID"].to_list()))
#      .then(None)
#      .otherwise(pl.col(c))
#      .alias(c)
#    for c in ["SamID", "MorID", "FarID"]
#])


In [9]:
# Henter inn fornavn og etternavn til den personen som er tilknyttet
# Denne listen benyttes til å koble alle personer i Cornerstone
# etter at alle personer er lagt inn.

# Perform the join by preparing the right side first
df_connectedMedlemer = df_connectedMedlemer.join(
    # 1. Select and rename columns from 'df' on the fly
    df.select(
        pl.col("MedlemID"),  # The join key must be included
        pl.col("FORNAVN").alias("Connected_FORNAVN"),
        pl.col("EFTERNAVN").alias("Connected_EFTERNAVN")
    ),
    left_on="ConnectedMedlemID",
    right_on="MedlemID",
    how="left"  # Using 'left' is safer to not lose rows from the left table
)
print(df_connectedMedlemer)

shape: (381, 7)
┌────────────────┬─────────────┬───────┬──────────┬────────────────┬───────────────┬───────────────┐
│ FORNAVN        ┆ EFTERNAVN   ┆ type  ┆ MedlemID ┆ ConnectedMedle ┆ Connected_FOR ┆ Connected_EFT │
│ ---            ┆ ---         ┆ ---   ┆ ---      ┆ mID            ┆ NAVN          ┆ ERNAVN        │
│ str            ┆ str         ┆ str   ┆ i64      ┆ ---            ┆ ---           ┆ ---           │
│                ┆             ┆       ┆          ┆ i64            ┆ str           ┆ str           │
╞════════════════╪═════════════╪═══════╪══════════╪════════════════╪═══════════════╪═══════════════╡
│ Andreas        ┆ Tangen      ┆ SamID ┆ 15628    ┆ 13248          ┆ Katarina      ┆ Lunde         │
│ Sissel         ┆ Monsen      ┆ SamID ┆ 15513    ┆ 15484          ┆ Svenn Olav    ┆ Kalvø         │
│ Marit          ┆ Brendbekken ┆ SamID ┆ 14476    ┆ 14477          ┆ Stein         ┆ Befring       │
│ Stein          ┆ Befring     ┆ SamID ┆ 14477    ┆ 14476          ┆ Marit 

In [10]:
# Samler sammen personer som vi antar har kavalitets data
# Til den første runden av konvertering
df_kvalitetsdataID = pl.concat([
    # Alle personer som har kontakter på seg
    df_connectedMedlemer["MedlemID"],
    # Alle personer som er en kontakt på en annen person
    df_connectedMedlemer["ConnectedMedlemID"],
    # Alle personer som har status som medlem
    df.filter(pl.col("Kategori").str.starts_with("M"))["MedlemID"]
]).unique().to_frame("MedlemID")
print(df_kvalitetsdataID)

shape: (408, 1)
┌──────────┐
│ MedlemID │
│ ---      │
│ i64      │
╞══════════╡
│ 11018    │
│ 11027    │
│ 11084    │
│ 11208    │
│ 11209    │
│ …        │
│ 15638    │
│ 15649    │
│ 15650    │
│ 15659    │
│ 15663    │
└──────────┘


In [11]:
# Lager en liste av personer som har god kvalitet på data
# One‑liner using a semi‑join (no need for unique() – keeps only df rows whose MedlemID is in the concatenated list)
df_kvalitetsData = df.join(
    df_kvalitetsdataID,
    on="MedlemID"
)
print(df_kvalitetsData)


# Lager en liste av personer som sjanse for dårligere kvalitet på data
# Option 1: Anti‑join (most idiomatic & performant)
df_looserData = df.join(
    df_kvalitetsData,
    on="MedlemID",
    how="anti"
)
print(df_looserData)

shape: (408, 46)
┌──────────┬────────────┬────────────┬──────────┬───┬────────────┬──────────┬─────────┬────────────┐
│ MedlemID ┆ Innmelding ┆ Er den     ┆ Menighet ┆ … ┆ UtmeldtDat ┆ DødsDato ┆ Kommune ┆ KID-nr     │
│ ---      ┆ sdato      ┆ innmeldte  ┆ ---      ┆   ┆ o          ┆ ---      ┆ ---     ┆ ---        │
│ i64      ┆ ---        ┆ døpt ?     ┆ str      ┆   ┆ ---        ┆ date     ┆ str     ┆ str        │
│          ┆ date       ┆ ---        ┆          ┆   ┆ str        ┆          ┆         ┆            │
│          ┆            ┆ bool       ┆          ┆   ┆            ┆          ┆         ┆            │
╞══════════╪════════════╪════════════╪══════════╪═══╪════════════╪══════════╪═════════╪════════════╡
│ 15628    ┆ null       ┆ false      ┆ Bergen   ┆ … ┆ null       ┆ null     ┆ null    ┆ null       │
│ 15621    ┆ null       ┆ false      ┆ null     ┆ … ┆ null       ┆ null     ┆ null    ┆ null       │
│ 15626    ┆ 2021-12-17 ┆ true       ┆ Bergen   ┆ … ┆ null       ┆ null   